In [255]:
# Import Meteostat library and dependencies
import os
import time
import scipy
import folium
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import MoveTargetOutOfBoundsException

from arcgis.geocoding import geocode
from arcgis.gis import GIS
from arcgis.geometry import Point


import numpy as np 
from glob import glob
import pandas as pd

import seaborn as sns 
import matplotlib.pyplot as plt 
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from googletrans import Translator

In [286]:
service = Service()
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(service=service, options=options)
# Find the canv
driver.get("https://aqicn.org/map/bangkok/")
# pe = driver.find_elements(By.XPATH, "//div[@id='map_canvas']")
pe = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, 
                    "//div[@id='map_canvas']")))
set_=driver.find_elements(By.XPATH, "//div[@class='leaflet-bar leaflet-control']")
a=set_[0].find_elements(By.TAG_NAME, "a")
a[0].click()
canva_ele = pe[0].find_elements(By.TAG_NAME, "canvas")
p = pe[0].find_elements(By.XPATH, "//div[@class='leaflet-pixi-overlay leaflet-zoom-animated']")
s = pe[0].find_elements(By.TAG_NAME, "svg")

driver.maximize_window()
driver.set_window_size(1200, 1200)

set_0 = driver.find_elements(By.XPATH , "//div[@class='leaflet-control-container']")
a_=set_0[0].find_elements(By.TAG_NAME, "a")
# print(a_)
# a_[0].click()

#find the div
actions = ActionChains(driver)
# driver.maximize_window()
# r=s[0].find_elements(By.XPATH, "//div[@id='leaflet-tooltip aqi-marker-tooltip aqi-marker-tooltip-desktop leaflet-zoom-animated leaflet-interactive leaflet-tooltip-left']")

# print(r[0].get_attribute("translate3d"))
height = canva_ele[0].get_attribute('height')
width = canva_ele[0].get_attribute('width')
actions.move_to_element_with_offset(s[0], int(height)//2, int(width)//2).perform()

map_width, map_height = int(width), int(height) 
x_ = 0
y_ = 0
imgs=driver.find_elements(By.XPATH ,"//div[@class='leaflet-tile-container leaflet-zoom-animated']")
imgs_=imgs[0].find_elements(By.TAG_NAME, 'img')
offset_x = 20  # Adjust as needed
offset_y = 20  # Adjust as needed
dic = {"pm25":[], "pixel_location":[], "img_num":[], "location":[]}
# driver.set_window_size(1280, 900)
l_ = ''
max_retries = 10
for image in imgs_:
        retry_count = 0
        location = image.location  # Get position as a dict with x and y
        size = image.size  # Get size as a dict with width and height
        print(f'Image Source: {image.get_attribute("src")}')
        _x = location["x"]
        _y = location["y"]
        img_height = int(size["height"])
        img_width = int(size["width"])
        print(f'Position: (X: {location["x"]}, Y: {location["y"]})')
        viewport_width_ = driver.execute_script("return window.innerWidth")
        viewport_height_ = driver.execute_script("return window.innerHeight")
        if (viewport_width_ > _x > 0 and  viewport_height_ > _y > 0):
            pe = driver.find_elements(By.XPATH, "//div[@id='map_canvas']")
            s = pe[0].find_elements(By.TAG_NAME, "svg")
            if _x <=map_width and _y <= map_height:
                actions.move_to_element_with_offset(s[0], _x + (size['width'] // 2), _y + (size['height'] // 2)).perform()

        viewport_width = driver.execute_script("return window.innerWidth")
        viewport_height = driver.execute_script("return window.innerHeight")
        print(viewport_width , viewport_height)
#         c = '0'
        for k,y in enumerate(range(_y, _y+img_height, offset_y)):
            for b,x in enumerate(range(_x, _x+img_width, offset_x)):
                #check for image_width and image_height with respect to view_port
                if _x <=map_width and _y <= map_height:

                    #relative condition
#                     if (0 < x < _x+img_width) & (0 < y < _y+img_height):
#                         print(f"x:{x}, y:{y}")
    #                     relative_x = x / img_width
    #                     relative_y = y / img_height
#                         print(img_width, img_height)
                        pe = driver.find_elements(By.XPATH, "//div[@id='map_canvas']")
#                         pe = WebDriverWait(driver, 40).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@id='map_canvas']")))
                        s = pe[0].find_elements(By.TAG_NAME, "svg")
                        g = s[0].find_elements(By.TAG_NAME, "g")  
                        path = g[0].find_elements(By.TAG_NAME, "path")
                        if len(path) == 2:
                            try:
                                o = pe[0].find_elements(By.XPATH , "//div[@class='leaflet-pane leaflet-tooltip-pane']")
                                r = o[0].find_elements(By.TAG_NAME, "div")
                                f = r[0].find_elements(By.TAG_NAME, "center")
                                table = f[0].find_elements(By.TAG_NAME, "table")
#                                 element = WebDriverWait(driver, 10).until(
#                                                 EC.visibility_of_element_located(
                                time.sleep(4)
                                l = o[0].find_element(By.XPATH, "//div[@class='label-aqistation-title']")
                                if len(table) > 0:
                                    driver.execute_script("arguments[0].scrollIntoView(true);", table[0])
                                    tbody = table[0].find_elements(By.TAG_NAME, "tbody")
        #                         trow = tbody[0].find_elements(By.TAG_NAME, "tr")
#                                     if b > 5 and table[0].text == c:
#                                         continue
                                 
#                                     c = table[0].text 
                                    P = table[0].text
                                    if l_ != l:
                                        dic["location"].append(l.text)
                                        dic["pm25"].append(table[0].text)
                                        dic["pixel_location"].append((x,  y))
                                        dic["img_num"].append(k)
                                    l_ = l
        
                            except StaleElementReferenceException:
                                pass
                            print('only' , img_height, img_width)
                            try:
                                if  _x<x < _x+img_width and  _y<y <  _y+img_height:
                                    actions.move_to_element_with_offset(s[0], x, y).perform()
                            except MoveTargetOutOfBoundsException:
                                print(f"MoveTargetOutOfBoundsException caught for coordinates ({x}, {y})")
            
                                # Optionally, adjust coordinates slightly (e.g., reduce by 10px)
                                x -= 10
                                y -= 10

                                # Retry the action with the adjusted coordinates
                                retry_count += 1
                                print(f"Retrying with adjusted coordinates: ({x}, {y})")

                                # Optional: Add a small delay before retrying
                                time.sleep(1)

                                if retry_count == max_retries:
                                    print("Maximum retry attempts reached, skipping this set of coordinates.")
                                    continue  # Skip to next iteration if retries exhausted


                            #extract all the values

driver.quit()

Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/944/1595
Position: (X: 478, Y: 268)
1200 968
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/945/1595
Position: (X: 478, Y: 533)
1200 968
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/944/1594
Position: (X: 213, Y: 268)
1200 968
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/944/1596
Position: (X: 743, Y: 268)
1200 968
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only

only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/946/1594
Position: (X: 213, Y: 798)
1200 968
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
only 265 265
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/946/1596
Position: (X: 743, Y: 798)
1200 968
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/944/1593
Position: (X: -52, Y: 268)
1200 968
Image Source: https://server.arc

Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/944/1599
Position: (X: 1538, Y: 268)
1200 968
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/943/1591
Position: (X: -582, Y: 3)
1200 968
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/943/1599
Position: (X: 1538, Y: 3)
1200 968
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/945/1591
Position: (X: -582, Y: 533)
1200 968
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/945/1599
Position: (X: 1538, Y: 533)
1200 968
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/942/1591
Position: (X: -582, Y: -262)
1200 968
Image Source: https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/11/942/1599
Position: (X: 1538

In [14]:

# # Initialize the Firefox driver
# service = Service()
# options = webdriver.FirefoxOptions()
# driver = webdriver.Firefox(service=service, options=options)
# dic = {"pm25":[], "pixel_location":[], "img_num":[], "location":[]}

# try:
#     driver.get("https://aqicn.org/map/bangkok/")
#     wait = WebDriverWait(driver, 20)
#     set_=driver.find_elements(By.XPATH, "//div[@class='leaflet-bar leaflet-control']")
#     set_[0].click()
#     # Find the canvas and related elements
#     pe = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='map_canvas']")))
#     canva_ele = pe.find_elements(By.TAG_NAME, "canvas")
#     s = pe.find_elements(By.XPATH, "//div[@class='leaflet-pixi-overlay leaflet-zoom-animated']")

#     # Set window size for better visibility
#     driver.set_window_size(1280, 900)

#     dic = {"pm25": [], "pixel_location": [], "img_num": [], "location": []}

#     imgs = driver.find_elements(By.XPATH, "//div[@class='leaflet-tile-container leaflet-zoom-animated']")
#     imgs_ = imgs[0].find_elements(By.TAG_NAME, 'img')

#     for k, image in enumerate(imgs_):
#         location = image.location
#         size = image.size
#         print(f'Image Source: {image.get_attribute("src")}')
        
#         _x = int(location["x"])
#         _y = int(location["y"])
#         print(f'Position: (X: {_x}, Y: {_y})')

#         # Ensure we are moving to the right element
#         actions = ActionChains(driver)
#         actions.move_to_element_with_offset(s[0], _x + (size['width'] // 2), _y + (size['height'] // 2)).perform()

#         # Adjust offsets for iteration
#         offset_x = 20
#         offset_y = 20
#         img_height = int(size["height"])
#         img_width = int(size["width"])

#         for y in range(_y, _y + img_height, offset_y):
#             for x in range(_x, _x + img_width, offset_x):
#                 if (0 < x < _x + img_width) and (0 < y < _y + img_height):
#                     print(f"x: {x}, y: {y}")

#                     # Re-fetch elements to avoid StaleElementReferenceException
#                     pe = driver.find_elements(By.XPATH, "//div[@id='map_canvas']")
#                     s = pe[0].find_elements(By.TAG_NAME, "svg")
#                     g = s[0].find_elements(By.TAG_NAME, "g")
#                     path = g[0].find_elements(By.TAG_NAME, "path")

#                     if len(path) > 1:
#                         try:
#                             o = pe[0].find_elements(By.XPATH, "//div[@class='leaflet-pane leaflet-tooltip-pane']")
#                             if o:
#                                 r = o[0].find_elements(By.TAG_NAME, "div")
#                                 f = r[0].find_elements(By.TAG_NAME, "center")
#                                 table = f[0].find_elements(By.TAG_NAME, "table")

#                                 if table:
#                                     driver.execute_script("arguments[0].scrollIntoView(true);", table[0])
#                                     tbody = table[0].find_elements(By.TAG_NAME, "tbody")

#                                     l = o[0].find_element(By.XPATH, "//div[@class='label-aqistation-title']")
#                                     dic["pm25"].append(table[0].text)
#                                     dic["pixel_location"].append((x + img_width, y + img_height))
#                                     dic["img_num"].append(k)
#                                     dic["location"].append(l.text)

#                         except StaleElementReferenceException:
#                             print("StaleElementReferenceException caught, skipping.")
                        
#                         # Move again to avoid any issues
#                         actions.move_to_element_with_offset(s[0], x, y).perform()

# finally:
#     driver.quit()

In [280]:
pd.Series(dic['pm25']).value_counts()

78 - Moderate    2
76 - Moderate    2
84 - Moderate    1
82 - Moderate    1
                 1
65 - Moderate    1
69 - Moderate    1
90 - Moderate    1
59 - Moderate    1
54 - Moderate    1
63 - Moderate    1
Name: count, dtype: int64

In [322]:
c=pd.DataFrame(dic)
c.head()

,pm25,pixel_location,img_num,location
0,76 - Moderate,"(598, 268)",0,X\nวัดอุบลวนาราม
1,119 - Unhealthy for sensitive groups,"(498, 533)",0,X\nโรงเรียนวัดราษฎร์บำรุง แขวงหลักสอง เขตบางแค...
2,84 - Moderate,"(743, 268)",0,X\nศูนย์พัฒนาเด็กเล็กเทศบาลเสาธงหิน 1 ต.เสาธงห...
3,78 - Moderate,"(763, 308)",2,X\nศพด.รร.ผาสุกมณีจักรมิตรภาพที่ 116 ทน.ปากเกร...
4,119 - Unhealthy for sensitive groups,"(903, 308)",2,X\nสวนสมเด็จพระศรีนครินทร์ ทน.ปากเกร็ด ต.บ้านใ...


In [302]:
location_ = c['location']

In [323]:
translator = Translator()

c['en'] = ''
for i,loc in enumerate(location_):
    translated_text = translator.translate(loc, src='th', dest='en')
    c.loc[i, 'en'] = str(translated_text.text)[2:]

In [324]:
c.head()

,pm25,pixel_location,img_num,location,en
0,76 - Moderate,"(598, 268)",0,X\nวัดอุบลวนาราม,Ubanaram Temple
1,119 - Unhealthy for sensitive groups,"(498, 533)",0,X\nโรงเรียนวัดราษฎร์บำรุง แขวงหลักสอง เขตบางแค...,"Wat Rat Bamrung School, Lak Song Subdistrict, ..."
2,84 - Moderate,"(743, 268)",0,X\nศูนย์พัฒนาเด็กเล็กเทศบาลเสาธงหิน 1 ต.เสาธงห...,"Child Development Center, Sao Thong Hin Munici..."
3,78 - Moderate,"(763, 308)",2,X\nศพด.รร.ผาสุกมณีจักรมิตรภาพที่ 116 ทน.ปากเกร...,"The funeral of the Pha Suksani School, Friends..."
4,119 - Unhealthy for sensitive groups,"(903, 308)",2,X\nสวนสมเด็จพระศรีนครินทร์ ทน.ปากเกร็ด ต.บ้านใ...,"Somdet Phra Srinakarin Park, Pak Kred, Ban Mai..."


In [325]:
def get_lat_long_by_state(location):
    if location  != None or location != '' :
        geolocator = Nominatim(user_agent="myGeopyApp/1.0")
        esrihq = geocode(location)
        if esrihq != None:
            latitude, longitude = esrihq[0]['location']['x'], esrihq[0]['location']['y']
            return latitude, longitude

In [333]:
#exctract the co-ordinates for all the location
c['latitude'] = 0
c['longitude'] = 0
gis = GIS()

for i,loc in enumerate(c['en']):
    latitude, longitude = get_lat_long_by_state(loc)
    c.loc[i,'latitude'] = latitude
    c.loc[i,'longitude'] = longitude


In [300]:
current_time = datetime.now()
np.save(f'f3_{current_time}.npy', dic)


In [334]:
c.head()

,pm25,pixel_location,img_num,location,en,latitude,longitude
0,76 - Moderate,"(598, 268)",0,X\nวัดอุบลวนาราม,Ubanaram Temple,-97.379027,31.092302
1,119 - Unhealthy for sensitive groups,"(498, 533)",0,X\nโรงเรียนวัดราษฎร์บำรุง แขวงหลักสอง เขตบางแค...,"Wat Rat Bamrung School, Lak Song Subdistrict, ...",100.377546,13.701472
2,84 - Moderate,"(743, 268)",0,X\nศูนย์พัฒนาเด็กเล็กเทศบาลเสาธงหิน 1 ต.เสาธงห...,"Child Development Center, Sao Thong Hin Munici...",100.401849,13.875027
3,78 - Moderate,"(763, 308)",2,X\nศพด.รร.ผาสุกมณีจักรมิตรภาพที่ 116 ทน.ปากเกร...,"The funeral of the Pha Suksani School, Friends...",100.498830,13.913010
4,119 - Unhealthy for sensitive groups,"(903, 308)",2,X\nสวนสมเด็จพระศรีนครินทร์ ทน.ปากเกร็ด ต.บ้านใ...,"Somdet Phra Srinakarin Park, Pak Kred, Ban Mai...",80.800703,6.996240


In [349]:
import re 
c['npm25']=c['pm25'].apply(lambda x : re.match(r'[0-9]*', x).group())
c['npm25']=c['npm25'].astype('int')

In [355]:
c

,pm25,pixel_location,img_num,location,en,latitude,longitude,npm25
0,76 - Moderate,"(598, 268)",0,X\nวัดอุบลวนาราม,Ubanaram Temple,-97.379027,31.092302,76
1,119 - Unhealthy for sensitive groups,"(498, 533)",0,X\nโรงเรียนวัดราษฎร์บำรุง แขวงหลักสอง เขตบางแค...,"Wat Rat Bamrung School, Lak Song Subdistrict, ...",100.377546,13.701472,119
2,84 - Moderate,"(743, 268)",0,X\nศูนย์พัฒนาเด็กเล็กเทศบาลเสาธงหิน 1 ต.เสาธงห...,"Child Development Center, Sao Thong Hin Munici...",100.401849,13.875027,84
3,78 - Moderate,"(763, 308)",2,X\nศพด.รร.ผาสุกมณีจักรมิตรภาพที่ 116 ทน.ปากเกร...,"The funeral of the Pha Suksani School, Friends...",100.498830,13.913010,78
4,119 - Unhealthy for sensitive groups,"(903, 308)",2,X\nสวนสมเด็จพระศรีนครินทร์ ทน.ปากเกร็ด ต.บ้านใ...,"Somdet Phra Srinakarin Park, Pak Kred, Ban Mai...",80.800703,6.996240,119
5,111 - Unhealthy for sensitive groups,"(453, 533)",0,X\nโรงเรียนตั้งพิรุฬห์ธรรม แขวงศาลาธรรมสพน์ เข...,Tang Phirun Dharma SchoolSala Thammasop Subdis...,100.353490,13.772991,111
6,65 - Moderate,"(763, 533)",0,X\nโรงเรียนประถมนนทรี เขตยานนาวา กทม.,"Nonsi Primary School, Yannawa District, Bangkok",100.552148,13.712718,65
7,71 - Moderate,"(803, 553)",1,X\nBangkok,Bangkok,100.502243,13.753958,71
8,52 - Moderate,"(963, 553)",1,X\nวัดกระโจมทอง,Golden Krathong Temple,-85.042311,33.845474,52
9,56 - Moderate,"(1003, 553)",1,X\nโรงเรียนอุดมศึกษา,Udom Suksa School,99.857886,12.211237,56


In [365]:
map_center = [13.7563, 100.5018]  # Coordinates for New York City
my_map = folium.Map(location=map_center, zoom_start=12)
# Add markers based on the value
for i,k in enumerate(range(c.shape[0])):
    color = "red" if c['npm25'].loc[i] >= 80 else "blue"  
    folium.Marker(
        location=[c['longitude'].loc[i], c['latitude'].loc[i]],
        popup=f"Value: {c['npm25'].loc[i]}",
        icon=folium.Icon(color=color)
    ).add_to(my_map)

# # Optionally, use CircleMarker to make a more prominent marker
# for lat, lon, value in locations:
#     color = "red" if value >= 80 else "blue"  # Customize the color condition
#     folium.CircleMarker(
#         location=[lat, lon],
#         radius=10,  # Adjust the size of the circle
#         popup=f"Value: {value}",
#         color=color,
#         fill=True,
#         fill_color=color
#     ).add_to(my_map)

# Save the map to an HTML file
# my_map.save("folium_map.html")

In [366]:
my_map